## Split by tournament

The following cell computes the elo with rating periods split by tournament. It will be MASSIVE, about 20,000 tournaments x 100,000 players and probably needs a few dozen GB of memory.

TODO: Print this line by line to a database instead to avoid absurd memory usage.

In [ ]:
# Get the start/end positions of each tournament in the sorted dataframe
# Stored as (start,end)
tournament_sets_positions = {}

# This is gonna take a while. Might as well show a progress bar
for i in tqdm.tqdm(range(0, len(sets_df.index)), total=len(sets_df.index)):
    key = sets_df.iloc[i]['tournament_key']
    if key not in tournament_sets_positions:
        tournament_sets_positions[key] = (i,i)
    else:
        tournament_sets_positions[key] = (tournament_sets_positions[key][0], i)

In [ ]:
# Let's get the dates of each period (tournament) and, importantly, know how many there are
# This will be useful for pre-allocating the entire pandas dataframe rather than constantly building it row by row
# NOTE: Some tournaments might have the same end date. That's ok.
#       The algorithm will simply update those entries multiple times with newer and newer glicko values.
tournament_list = [x for x in tournament_sets_positions]
dates = list(set([sets_df.iloc[tournament_sets_positions[tournament][0]]['end'] for tournament in tournament_list])) # VERY much need these to be unique
dates.sort() # Just to be absolutely 100% sure, but it probably stays sorted with the above operation anyways

unique_players = list(set(list(sets_df['p1_id'].unique()) + list(sets_df['p2_id'].unique())))

# Convenient store of glicko objects
glicko_objects = {}
for player in unique_players:
    glicko_objects[player] = Player()

print(len(unique_players))
print(len(dates))

print("This takes LOADS of memory.")
raise KeyboardInterrupt

# Initial row of default ELO values, at the earliest possible date.
# Note that we are also pre-allocating the dataframe with the rest of the possible dates.
player_ratings_df = pd.DataFrame([[1500.0] * len(unique_players)], columns=unique_players, index=[datetime.datetime(2015, 1, 11, 14, 16, 0)] + dates)
player_rds_df = pd.DataFrame([[350.0] * len(unique_players)], columns=unique_players, index=[datetime.datetime(2015, 1, 11, 14, 16, 0)] + dates)

# Loop over tournaments
# NOTE: Now we have to only consider tournaments that actually have sets
for tournament_key in tqdm.tqdm(tournament_list, total=len(tournament_list)):
    process_tournament(player_ratings_df, player_rds_df, sets_df, tournament_sets_positions, tournament_key, False, glicko_objects)